In [18]:
import pandas as pd
import numpy as np
import math
import datetime
pd.set_option('display.max_columns', 40)
pd.set_option('display.max_rows', 4)
van_total_pop = 675.218

In [19]:
df = pd.read_csv("Vancouver.csv")
df = df[df.YEAR >= 2015] # Remove rows with years less than 2015
df = df.reset_index(drop=True) # Resets the index

In [20]:
df.tail(5)

,TYPE,YEAR,MONTH,DAY,HOUR,MINUTE,HUNDRED_BLOCK,NEIGHBOURHOOD,X,Y
193373,Vehicle Collision or Pedestrian Struck (with I...,2018,8,27,17,33,YUKON ST / W 4TH AVE,Mount Pleasant,491800.0,5457199.0
193374,Vehicle Collision or Pedestrian Struck (with I...,2016,4,20,19,18,YUKON ST / W 5TH AVE,Mount Pleasant,491797.0,5457103.0
...,...,...,...,...,...,...,...,...,...,...
193376,Vehicle Collision or Pedestrian Struck (with I...,2018,8,4,16,22,YUKON ST / W BROADWAY AVE,Mount Pleasant,491786.0,5456719.0
193377,Vehicle Collision or Pedestrian Struck (with I...,2016,6,20,18,11,YUKON ST / W KING EDWARD AVE,Riley Park,491786.0,5455143.0


In [21]:
# Converting X and Y to lat and lon

R = 6360000 # earth radius

# Formula to convert X and Y to lat an lon
def cc_to_ll(x, y):
    (x, y) = x, y # does not use `z`
    r = math.sqrt(x**2 + y**2)
    long =180 * math.atan2(y,x)/math.pi
    lat = 180 * math.acos(r/R)/math.pi
    return (lat, long)

# Gets all the X and Y columns as lists
X = df['X'].tolist()
Y = df['Y'].tolist()

lat = []
lon = []
for i in range(df.shape[0]):
    latitude, longitude = cc_to_ll(X[i], Y[i])
    lat.append(latitude)
    lon.append(longitude)

# Drops the old X and Y columns
df.drop(['X', 'Y'], axis=1, inplace=True)

# Adds the new lat and lon columns
df['Latitude'] = latitude
df['Longitude'] = longitude

In [22]:
# Organizing columns
df.rename(columns={'HUNDRED_BLOCK': 'Location-name', 'NEIGHBOURHOOD': 'Neighborhood'}, inplace=True) # Renaming wrong names
df['City'] = 'Vancouver' # Adding the city column
df.drop(['TYPE', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE'], axis=1, inplace=True) # Dropping useless columns to this table
df['Location-key'] = df.index # Creates Date_Key column based on index
df["Location-key"] += 200 # Adds 200 to every row in the column
cols = ["Location-key", "Location-name", "Longitude", "Latitude", 
        "Neighborhood", "City"]
crime_rate = df.shape[0] / van_total_pop * 100000 # Creating the crime_rate list of values
df = df[cols] # Reorganize the columns
df['Crime-rate'] = crime_rate # Create the crime-rate column

In [23]:
census_df = pd.read_csv("Census_Vancouver_Changed.csv")
data_neigh = {} # Dictonary of key neighborhood and value as census following the order of the dataframe
for column in census_df:
    data_neigh[column] = census_df[column].tolist()
del data_neigh['Unnamed: 0'] # Delete uncessary column

In [24]:
table = [] # Table containing a list of lists, where list[0] is the values of data_neigh correspoding to df original order
n_columns = len(census_df.iloc[:, 0].tolist())
for i in range(df.shape[0]):
    neigh = df['Neighborhood'][i]
    if neigh in data_neigh.keys():
        table.append(data_neigh[neigh])
    else:
        table.append(['Nan'] * n_columns)
neigh_df = pd.DataFrame(table, columns=census_df.iloc[:, 0].tolist())

In [25]:
df = pd.concat([df, neigh_df], axis=1) # Concatenates side-by-side the df and neigh_df dataframes 

In [26]:
df = df.rename(columns={'Total population by age groups': 'Total-neighborhood-population', 
                       '   0 to 4 years ': 'Years-0-to-4 ', '   5 to 9 years ': 'Years-5-to-9',
                       '   10 to 14 years ': 'Years-10-to-14','   15 to 19 years ': 'Years-15-to-19', 
                       '     15 years ': 'Years-15', '     16 years ': 'Years-16', '     17 years ': 'Years-17',
                       '     18 years ': 'Years-18 ', '     19 years ': 'Years-19', '   20 to 24 years ': 'Years-20-to-24',
                       '   25 to 29 years ': 'Years-25-to-29', '   30 to 34 years ': 'Years-30-to-34',
                       '   35 to 39 years ': 'Years-35-to-39', '   40 to 44 years ': 'Years-40-to-44',
                       '   45 to 49 years ': 'Years-45-to-49', '   50 to 54 years ': 'Years-50-to-54',
                       '   55 to 59 years ': 'Years-55-to-59', '   60 to 64 years ': 'Years-60-to-64',
                       '   65 to 69 years ': 'Years-65-to-69', '   70 to 74 years ': 'Years-70-to-74',
                       '   75 to 79 years ': 'Years-75-to-79', '   80 to 84 years ': 'Years-80-to-84',
                       '  85 years and over': 'Years-85-plus'}) # Formats names

In [27]:
df

,Location-key,Location-name,Longitude,Latitude,Neighborhood,City,Crime-rate,Total-neighborhood-population,Years-0-to-4,Years-5-to-9,Years-10-to-14,Years-15-to-19,Years-15,Years-16,Years-17,Years-18,Years-19,Years-20-to-24,Years-25-to-29,Years-30-to-34,Years-35-to-39,Years-40-to-44,Years-45-to-49,Years-50-to-54,Years-55-to-59,Years-60-to-64,Years-65-to-69,Years-70-to-74,Years-75-to-79,Years-80-to-84,Years-85-plus
0,200,10XX SITKA SQ,84.84866,30.547817,Fairview,Vancouver,2.863934e+07,"31,445","1,080",615,525,615,95,110,100,140,170,"1,770","4,120","4,005","2,980","2,665","2,275","2,105","1,995","1,995","1,405",955,785,665,880
1,201,10XX ALBERNI ST,84.84866,30.547817,West End,Vancouver,2.863934e+07,"44,540",960,465,325,760,85,80,110,170,315,"3,905","7,110","5,910","4,295","3,640","3,410","2,985","2,745","2,465","1,900","1,295",895,760,720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193376,193576,YUKON ST / W BROADWAY AVE,84.84866,30.547817,Mount Pleasant,Vancouver,2.863934e+07,"26,400","1,180",695,630,620,130,115,105,120,145,"1,765","3,815","3,850","3,080","2,355","2,035","1,690","1,445","1,220",640,505,350,255,275
193377,193577,YUKON ST / W KING EDWARD AVE,84.84866,30.547817,Riley Park,Vancouver,2.863934e+07,"21,795","1,195","1,105",990,980,210,180,210,190,185,"1,335","1,830","1,870","1,765","1,870","1,850","1,590","1,410","1,260",730,635,610,430,340


In [28]:
df.to_csv("Vancouver_Location_Final.csv", index=False) # Creates the csv file